In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [17]:
#data import
train=pd.read_csv("G:\\내 드라이브\\Lab\code\\workplace\\Study\\DL\\TabNet\\data\\atc\\pivot.csv")
train

,jobno,CR1_mean,CR1_min,CR1_max,CR1_std,CR1_var,ER1_mean,ER1_min,ER1_max,ER1_std,...,ST1_min,ST1_max,ST1_std,ST1_var,ST2_mean,ST2_min,ST2_max,ST2_std,ST2_var,prod_grade
0,1,44.460927,6.0,245.0,15.831372,250.632326,46.216444,1.0,100.0,24.897475,...,70.0,70.0,0.0,0.0,80.0,50.0,95.0,25.980762,675.0,A
1,2,58.524277,28.0,92.0,20.107806,404.323868,49.544407,1.0,100.0,21.936792,...,70.0,70.0,0.0,0.0,80.0,50.0,95.0,25.980762,675.0,A-
2,4,50.807634,25.0,88.0,18.928483,358.287479,42.368451,0.0,100.0,26.104542,...,70.0,70.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,B-
3,5,45.077235,24.0,56.0,8.947864,80.064275,45.367722,1.0,100.0,24.812140,...,70.0,70.0,0.0,0.0,72.5,50.0,95.0,31.819805,1012.5,A-
4,6,55.632718,25.0,86.0,17.811346,317.244039,48.017112,1.0,100.0,21.850776,...,70.0,70.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,137,53.955195,23.0,86.0,19.814728,392.623441,57.329167,0.0,100.0,22.657178,...,60.0,60.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,A
113,138,55.753194,23.0,95.0,21.557853,464.741016,52.681535,0.0,100.0,18.061244,...,60.0,60.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,A
114,139,46.465132,22.0,94.0,16.210828,262.790941,43.477958,0.0,100.0,24.163364,...,60.0,60.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,A
115,140,57.077212,24.0,157.0,21.608106,466.910264,53.576627,0.0,100.0,17.602888,...,60.0,60.0,0.0,0.0,95.0,95.0,95.0,0.000000,0.0,A


In [3]:
encoder=LabelEncoder()
categorical_dims={}

for col in train.columns:
    train[col]=encoder.fit_transform(train[col].values)
    categorical_dims[col]=train[col].nunique()
print(categorical_dims)

{'jobno': 117, 'CR1_mean': 116, 'CR1_min': 27, 'CR1_max': 41, 'CR1_std': 116, 'CR1_var': 116, 'ER1_mean': 116, 'ER1_min': 8, 'ER1_max': 8, 'ER1_std': 116, 'ER1_var': 116, 'ST1_mean': 3, 'ST1_min': 3, 'ST1_max': 2, 'ST1_std': 2, 'ST1_var': 2, 'ST2_mean': 4, 'ST2_min': 3, 'ST2_max': 1, 'ST2_std': 4, 'ST2_var': 4, 'prod_grade': 4}


In [4]:
features=train.columns.tolist()
features.remove('prod_grade')
print(len(features))
print(features)
cat_idxs=[]
for i,x in enumerate(features):
    cat_idxs.append(i)


cat_dims=[]
for v in categorical_dims.values():
    cat_dims.append(v)

cat_dims=cat_dims[:-1]
print(cat_dims)
print(len(cat_dims))

21
['jobno', 'CR1_mean', 'CR1_min', 'CR1_max', 'CR1_std', 'CR1_var', 'ER1_mean', 'ER1_min', 'ER1_max', 'ER1_std', 'ER1_var', 'ST1_mean', 'ST1_min', 'ST1_max', 'ST1_std', 'ST1_var', 'ST2_mean', 'ST2_min', 'ST2_max', 'ST2_std', 'ST2_var']
[117, 116, 27, 41, 116, 116, 116, 8, 8, 116, 116, 3, 3, 2, 2, 2, 4, 3, 1, 4, 4]
21


In [5]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
                      )

Device used : cuda


In [6]:
from sklearn.model_selection import train_test_split

X=train.drop(['prod_grade'],axis=1)
Y=train['prod_grade']
print(Y)

X=X.to_numpy()
Y=Y.to_numpy()

X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.1, shuffle=True, random_state=1004)
X_train, X_val, y_train, y_val=train_test_split(X_train,y_train,test_size=0.1, shuffle=True, random_state=1004)

print(X_train)

0      0
1      1
2      3
3      1
4      0
      ..
112    0
113    0
114    0
115    0
116    0
Name: prod_grade, Length: 117, dtype: int32
[[ 91  72  13 ...   0   0   0]
 [ 98  60  14 ...   0   0   0]
 [ 81  32   9 ...   0   0   0]
 ...
 [ 92 115  17 ...   0   0   0]
 [ 74   6   6 ...   0   0   0]
 [  3  40  22 ...   0   2   2]]


In [7]:
y_test

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
max_epochs = 100 if not os.getenv("CI", False) else 2

In [9]:
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
    )
    save_history.append(clf.history["valid_accuracy"])
    
#assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 2.1892  | train_accuracy: 0.32979 | valid_accuracy: 0.36364 |  0:00:01s
epoch 1  | loss: 1.71763 | train_accuracy: 0.37234 | valid_accuracy: 0.45455 |  0:00:01s
epoch 2  | loss: 1.59906 | train_accuracy: 0.32979 | valid_accuracy: 0.45455 |  0:00:01s
epoch 3  | loss: 1.41191 | train_accuracy: 0.32979 | valid_accuracy: 0.27273 |  0:00:01s
epoch 4  | loss: 1.20379 | train_accuracy: 0.31915 | valid_accuracy: 0.18182 |  0:00:01s
epoch 5  | loss: 1.24391 | train_accuracy: 0.37234 | valid_accuracy: 0.36364 |  0:00:01s
epoch 6  | loss: 1.15698 | train_accuracy: 0.40426 | valid_accuracy: 0.27273 |  0:00:01s
epoch 7  | loss: 1.01449 | train_accuracy: 0.45745 | valid_accuracy: 0.36364 |  0:00:01s
epoch 8  | loss: 0.94755 | train_accuracy: 0.51064 | valid_accuracy: 0.45455 |  0:00:01s
epoch 9  | loss: 0.87232 | train_accuracy: 0.44681 | valid_accuracy: 0.36364 |  0:00:01s
epoch 10 | loss: 0.85156 | train_accuracy: 0.45745 | valid_accuracy: 0.36364 |  0:00:01s
epoch 11 | loss: 0.88

In [10]:
import inspect

inspect.getsource(TabNetClassifier)

'class TabNetClassifier(TabModel):\n    def __post_init__(self):\n        super(TabNetClassifier, self).__post_init__()\n        self._task = \'classification\'\n        self._default_loss = torch.nn.functional.cross_entropy\n        self._default_metric = \'accuracy\'\n\n    def weight_updater(self, weights):\n        """\n        Updates weights dictionary according to target_mapper.\n\n        Parameters\n        ----------\n        weights : bool or dict\n            Given weights for balancing training.\n\n        Returns\n        -------\n        bool or dict\n            Same bool if weights are bool, updated dict otherwise.\n\n        """\n        if isinstance(weights, int):\n            return weights\n        elif isinstance(weights, dict):\n            return {self.target_mapper[key]: value for key, value in weights.items()}\n        else:\n            return weights\n\n    def prepare_target(self, y):\n        return np.vectorize(self.target_mapper.get)(y)\n\n    def compu

In [11]:
tab_preds = clf.predict_proba(X_test)

In [12]:
pred = np.clip(clf.predict(X_test), a_min=0, a_max=None)

In [13]:
tab_preds

array([[0.3706972 , 0.18854319, 0.26932365, 0.17143601],
       [0.36162755, 0.2758174 , 0.196757  , 0.16579801],
       [0.3504552 , 0.25773442, 0.20714433, 0.18466604],
       [0.3329279 , 0.22734943, 0.23135552, 0.20836714],
       [0.38584656, 0.28838763, 0.18262425, 0.14314157],
       [0.33637583, 0.26669964, 0.21311566, 0.18380883],
       [0.50446576, 0.19304134, 0.19928128, 0.10321161],
       [0.2969648 , 0.3071926 , 0.19854328, 0.19729929],
       [0.14732876, 0.49978295, 0.13377495, 0.21911334],
       [0.38320157, 0.27576166, 0.19136848, 0.14966834],
       [0.45049116, 0.48952323, 0.03631143, 0.02367421],
       [0.44024584, 0.18026043, 0.22523406, 0.15425968]], dtype=float32)

In [14]:
Y

array([0, 1, 3, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 3, 0, 3, 1, 3,
       0, 0, 2, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       0, 0, 0, 0, 0, 0, 0])

In [15]:
from sklearn.metrics import classification_report
report = classification_report(y_test, pred)
print(">> 모델 평가 리포트")
print(report)

>> 모델 평가 리포트
              precision    recall  f1-score   support

           0       0.89      0.73      0.80        11
           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1

    accuracy                           0.67        12
   macro avg       0.30      0.24      0.27        12
weighted avg       0.81      0.67      0.73        12



C:\Users\sohyang\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sohyang\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sohyang\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sohyang\anac

In [16]:
X_test

array([[ 54,   0,   3,   6,  42,  42,   5,   0,   6,  94,  94,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 93, 103,  14,  40, 104, 104,  82,   0,   6,  42,  42,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 17,  30,  18,   9,   2,   2,  32,   0,   6,  56,  56,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 76,  75,   7,  11,  77,  77,  65,   0,   6,  90,  90,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 79,  58,  10,  21,  53,  53,  72,   0,   6,  60,  60,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 88,  79,  11,  19,  80,  80,  51,   0,   6,  53,  53,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 38,   2,   7,  18,  44,  44,  11,   0,   6, 100, 100,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 78,  23,   9,   7,  19,  19,  61,   1,   6,  27,  27,   1,   1,
          0,   0,   0,   3,   2,   0,   0,   0],
       [ 94,  16,  12,  